# Analyzing CIA Factbook Data Using SQL

In this project, we use SQL to explore the database 'Factbook.db' from the CIA World Factbook which contains a statistical summary of all the countries. In order to connect our Jupyter Notebook to the database, one need to write the following program.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

Let us first print the first few rows to get acquainted with the database file.

In [2]:
%%sql
SELECT * FROM facts limit 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Now we can make some basic summary statistics. The description of some columns is as follows:

+) name - The name of the country.

+) area - The total land and sea area of the country.

+) population - The country's population.

+) population_growth- The country's population growth as a percentage.

+) birth_rate - The country's birth rate, or the number of births a year per 1,000 people.

+) death_rate - The country's death rate, or the number of death a year per 1,000 people.

+) area- The country's total area (both land and water).

+) area_land - The country's land area in square kilometers.

+) area_water - The country's waterarea in square kilometers.

In [15]:
%%sql
SELECT
    MIN(population) min_pop,# minimum polulation of the database
    MAX(population) max_pop, #maximum polulation
    MIN(population_growth) min_pop_grwth, 
    MAX(population_growth) max_pop_grwth 
FROM facts;


 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_grwth,max_pop_grwth
0,7256490011,0.0,4.02


As we can see, it is very strange that there is a country with population 0 and another one with population more than 7 billions. Let us look closer to see what is wrong by calling these countries.

In [19]:
%%sql
Select name From facts
Where population = (Select Min(population) From facts)

 * sqlite:///factbook.db
Done.


name
Antarctica


In [26]:
%%sql
Select name From facts
Where population = (Select Max(population) From facts)

 * sqlite:///factbook.db
Done.


name
World


So the country having more than 7 billions polulation is actually the whole world, which is reasonable. The country Antarctica does not have any inhabitant indeed.

Let us explore more information.

In [32]:
%%sql
Select Avg(population), Avg(area) From facts 

 * sqlite:///factbook.db
Done.


Avg(population),Avg(area)
62094928.32231405,555093.546184739


Next step is to determine all countries having more population than average but area smaller than average. These countries are densely populated than the rest. 

In [35]:
%%sql
Select name, population, area From facts
Where population > (Select Avg(population) From facts) And
area<(Select Avg(area) From facts)

 * sqlite:///factbook.db
Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Japan,126919659,377915
Philippines,100998376,300000
Thailand,67976405,513120
United Kingdom,64088222,243610
Vietnam,94348835,331210


We now want to see which countries having more water than land.

In [36]:
%%sql
Select name, area_land, area_water From facts
Where area_water > area_land 

 * sqlite:///factbook.db
Done.


name,area_land,area_water
British Indian Ocean Territory,60,54340
Virgin Islands,346,1564


Next question: Which countries will add the most people to their population next year? The answer is:

In [44]:
%%sql
Select name, population*population_growth From facts
Order By population*population_growth Desc
Limit 5

 * sqlite:///factbook.db
Done.


name,population*population_growth
World,7837009211.88
India,1527068612.48
China,615368424.6
Nigeria,444827037.20000005
Pakistan,290665336.62


Final question: Which countries have a higher death rate than birth rate? The answer is:

In [47]:
%%sql
Select name, birth_rate, death_rate From facts
Where death_rate > birth_rate
Limit 5

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
